In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

d:\ProgramData\Anaconda3\envs\rwn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
cora = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

In [20]:
print(cora[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [42]:

# Basic Graph Convolutional Network
class GCN(torch.nn.Module):
    def __init__(self, n_features, n_classes, n_channels=32, seed=29034875):
        super().__init__()
        torch.manual_seed(seed)
        self.conv1 = GCNConv(n_features, n_channels)
        self.conv2 = GCNConv(n_channels, n_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


def train_epoch(model, optimizer, criterion, dataset):
    model.train()
    optimizer.zero_grad()
    out = model(dataset.x, dataset.edge_index)
    loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()
    pred = out.argmax(dim=1)
    acc = pred[dataset.train_mask] == dataset.y[dataset.train_mask]
    acc = acc.sum() / len(acc)
    return loss, acc

def test(model, dataset):
    model.eval()
    out = model(dataset.x, dataset.edge_index)
    pred = out.argmax(dim=1)
    correct = pred[dataset.test_mask] == dataset.y[dataset.test_mask]
    acc = correct.sum() / dataset.test_mask.sum()

    return acc

In [43]:
dataset = cora[0]
model = GCN(cora.num_features, cora.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4) # Must be tuned
criterion = torch.nn.CrossEntropyLoss()

In [44]:
# Train for 100 epochs
print(' Epoch  |  Loss  | Acc')
print('-----------------------')
for e in range(100):
    l, acc = train_epoch(model, optimizer, criterion, dataset)
    print(f'  {e:3d}  |  {l:-.4f}  |  {acc:0.2f}')


 Epoch  |  Loss  | Acc
-----------------------
    0  |  1.9463  |  0.14
    1  |  1.9391  |  0.26
    2  |  1.9308  |  0.46
    3  |  1.9184  |  0.67
    4  |  1.9069  |  0.60
    5  |  1.8950  |  0.64
    6  |  1.8853  |  0.67
    7  |  1.8653  |  0.79
    8  |  1.8508  |  0.81
    9  |  1.8341  |  0.76
   10  |  1.8164  |  0.78
   11  |  1.7934  |  0.81
   12  |  1.7713  |  0.84
   13  |  1.7647  |  0.81
   14  |  1.7402  |  0.77
   15  |  1.7126  |  0.86
   16  |  1.6789  |  0.87
   17  |  1.6667  |  0.88
   18  |  1.6306  |  0.90
   19  |  1.6145  |  0.93
   20  |  1.5841  |  0.89
   21  |  1.5485  |  0.86
   22  |  1.5322  |  0.88
   23  |  1.5065  |  0.90
   24  |  1.4835  |  0.88
   25  |  1.4621  |  0.89
   26  |  1.4184  |  0.89
   27  |  1.3812  |  0.92
   28  |  1.3346  |  0.89
   29  |  1.3207  |  0.89
   30  |  1.2985  |  0.90
   31  |  1.2605  |  0.88
   32  |  1.2286  |  0.94
   33  |  1.2083  |  0.92
   34  |  1.1449  |  0.94
   35  |  1.1525  |  0.93
   36  |  1.1285 

In [45]:
# Get test acc
acc = test(model, dataset)
print(f'Test Accuracy: {acc:.2f}')

Test Accuracy: 0.81
